# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [3]:
import multiprocessing as mp
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re, warnings
warnings.filterwarnings(action='ignore')
url = 'https://en.wikipedia.org/wiki/Data_science'

In [4]:
html=requests.get(url).content
res=requests.get(url)
soup=bs(html,'html.parser')

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [5]:
uniq=[]
cont=soup.find_all('a')
#cont1=cont.get('href')
for e in range(3,len(cont)):
    uniq.append(cont[e].get('href')) 

In [6]:
print(len(uniq))

516


## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
domain = 'http://wikipedia.org'

In [8]:
abs=[e.replace("%","") for e in uniq if e.startswith('https')]
rel=[domain+f.replace("%","") for f in uniq if f.startswith('/wiki')]
webpag=set(abs+rel)
print(len(webpag))

258


## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [9]:
import os
print ('Numero de procesadores: ', mp.cpu_count())

Numero de procesadores:  2


In [11]:
access_rights = 0o755
os.mkdir('wikipedia',access_rights)
os.chdir('wikipedia')

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [12]:
from slugify import slugify

In [13]:
def index_page(link):
    try:
        open(slugify(link)+'.html','w+').write(requests.get(link).content)
    except:   
        pass
    
    
    

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [14]:
%%time
for e in webpag:
    index_page(e)

CPU times: user 19.8 s, sys: 908 ms, total: 20.7 s
Wall time: 2h 22min 32s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
import multiprocessing

In [ ]:
%%time
pool = multiprocessing.Pool()
result = pool.map(index_page,webpag)
pool.terminate()
pool.join()